In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,524 kB]
Hit:13 http://ppa.laun

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-16 16:28:21--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.38MB/s    in 0.2s    

2022-12-16 16:28:21 (5.38 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

Del 2: Determine Bias of Vine Reviews

In [9]:
# 1. Filter the data to retrieve all the rows where the total_votes count is >= 20
from pyspark.sql.functions import col

total_votes_df = vine_df.filter(col("total_votes") >= 20)

print(total_votes_df.count())
total_votes_df.show()

65379
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N

In [16]:
# 2. Using Step 1's DF, create a new DF to retrieve all the rows where the helpful_votes / total_votes >= 50% 
percent_helpful_votes_df = total_votes_df.withColumn('percent_helpful_votes', col("helpful_votes")/col("total_votes"))
percent_helpful_votes_df = percent_helpful_votes_df.filter(col("percent_helpful_votes") >= 0.5)
percent_helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|   0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|                  0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|                 0.84|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|   0.9166666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|   0.7846153846153846|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|   0.8611111111111112|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|   0.5588235294117647|


In [19]:
# 3. Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid)
vine_paid_df = percent_helpful_votes_df.filter(col("vine") == "Y")

print(vine_paid_df.count())
vine_paid_df.show()

94
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|   0.8888888888888888|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|                  1.0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|   0.7916666666666666|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|   0.8461538461538461|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|   0.7692307692307693|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|   0.9019607843137255|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|   0.785714285714285

In [21]:
# 4. Create a new DataFrame or table that retrieves all the rows where a review was not written as part of the Vine program (unpaid)
vine_not_paid_df = percent_helpful_votes_df.filter(col("vine") == "N")

print(vine_not_paid_df.count())
vine_not_paid_df.show()

40471
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|   0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|                  0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|                 0.84|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|   0.9166666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|   0.7846153846153846|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|   0.8611111111111112|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|   0.558823529411

In [38]:
# 5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

from pyspark.sql.functions import count,when,lit

vine_summary_df = percent_helpful_votes_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating")==5, True)).alias("Total_5_Star_Reviews"),
    (((count(when(col("star_rating")==5, True)))/(count(col("vine"))))*100).alias("5_Star_Percentage")
    )

vine_summary_df.show()

+----+-------------+--------------------+------------------+
|vine|Total_Reviews|Total_5_Star_Reviews| 5_Star_Percentage|
+----+-------------+--------------------+------------------+
|   Y|           94|                  48| 51.06382978723404|
|   N|        40471|               15663|38.701786464381904|
+----+-------------+--------------------+------------------+

